# **Impact Sécheresse - Restriction eau**

## **Librairies**

In [54]:
import io
import requests
import warnings
import pandas as pd
import numpy as np 
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import re
from unidecode import unidecode

warnings.filterwarnings("ignore")

## **Utils**

In [55]:
url_dep = "http://osm13.openstreetmap.fr/~cquest/openfla/export/departements-20140306-5m-shp.zip"

## 2023
zone_alerte_commune_url = (
    "https://www.data.gouv.fr/fr/datasets/r/25cfc138-313e-4e41-8eca-d13e3b04ca62"
)

restriction_guide_secheresse_url = (
    "https://www.data.gouv.fr/fr/datasets/r/b6e9da4a-d9e1-4854-8ea1-a87b86a0a5ca"
)

restrictions_url = (
    "https://www.data.gouv.fr/fr/datasets/r/07ebbc43-ea41-4e6d-a21c-58cedebe1320"
)

dataset_id_for_year = {
    2023: '782aac32-29c8-4b66-b231-ab4c3005f574',
    2022: '0fee8de1-c6de-4334-8daf-654549e53988',
    2021: 'c23fe783-763f-4669-a9b7-9d1d199fcfcd',
    2020: 'd16ae5b1-6666-4caa-930c-7993c4cd4188',
    2019: 'ed2e6cfa-1fe7-40a6-95bb-d9e6f99a78a0',
    2018: '8ba1889e-5496-47a6-8bf3-9371086dd65c',
    2017: 'ab886886-9b64-47ca-8604-49c9910c0b74',
    2016: 'fbd87d0b-a504-49e2-be6e-66a96ca4e489',
    2015: '98cb1f80-f296-4eae-a0b3-f236fc0b9325',
    2014: 'c68362d9-93ff-46bc-99a6-35d506855dae',
    2013: 'f9c1da33-19f4-499d-88cc-b3c247484215',
    2012: '43864992-e79b-449e-9d7d-93dad9b9df59',
    2011: '227149be-cd8b-4e59-a1a9-0840ef7f0a24',
    2010:'d6cb1826-6cc8-4709-85fd-433db23aa951'
}

In [102]:
def clean_column_names(column_names):
    # Replace spaces with underscores and remove special characters
    clean_names = [unidecode(re.sub(r'\W+', '', name.replace(' ', '_').lower())) for name in column_names]
    
    return clean_names

def clean_data(df):
    
    # columns selection
    selected_columns = ['Code INSEE', 'Code Postal', 'Commune', 'Département', 'Région']
    df = df[selected_columns]
    
    # Clean the column names
    df.columns = clean_column_names(df.columns)
    columns = df.columns
    
    for col in columns:
        df[col] = df[col].str.strip("[]'").astype(str)
        df[col] = df[col].str.strip('"').astype(str)       
    return df

def prepare_final_dataframe(dataframes, df_corres_commune_insee, id_zone_to_code_commune, filter_type):
    # List of years you want to clean
    # Dictionary to store cleaned data for each year
    cleaned_data = []
    
    # Load and clean data for each year
    for year, df in dataframes.items():
        cleaned_data.append(df)
    
    df = pd.concat(cleaned_data).reset_index(drop=True)
    df.drop_duplicates(inplace=True)

    # columns selection
    selected_columns = ['id_arrete', 'id_zone', 'debut_validite_arrete', 'fin_validite_arrete', 'numero_niveau', 'nom_niveau']
    df = df[selected_columns]
    nom_niveau_to_num_niveau = {"Pas de restriction": 0, "Vigilance": 1, "Alerte": 2, "Alerte renforcée": 3, "Crise": 4}
    df['numero_niveau'] = df['nom_niveau'].map(nom_niveau_to_num_niveau).fillna(np.nan)
    df.dropna(inplace=True)
    df['numero_niveau'] = df['numero_niveau'].astype(int)
    
    for col in ['debut_validite_arrete', 'fin_validite_arrete']:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    
    df = df[df.debut_validite_arrete<=df.fin_validite_arrete] # to check if we delete or use an other thin
    
    df['code_insee'] = df['id_zone'].map(id_zone_to_code_commune).fillna(np.nan)
    #df.sort_values(by=['code_insee', 'id_zone', 'numero_niveau'], inplace=True)
    #df.reset_index(drop=True, inplace=True)
    num_niveau_to_nom_niveau = df.set_index('numero_niveau')['nom_niveau'].to_dict()
    
    # Create a new DataFrame with a row for each day in each range
    dateranges = df.apply(lambda row: pd.date_range(row['debut_validite_arrete'], row['fin_validite_arrete']), axis=1)
    dateranges = dateranges.explode().reset_index(name='date')
    # Merge the expanded dateranges back with the original dataframe
    expanded_df = pd.merge(dateranges, df, left_on='index', right_index=True)
    
    # Drop the extra 'index' column and any other columns you don't need
    expanded_df = expanded_df.drop(columns=['index'])
    expanded_df.drop_duplicates(inplace=True)

    df_final = pd.merge(expanded_df, df_corres_commune_insee, on='code_insee', how='inner')
    df_final = df_final.groupby([filter_type, 'date'])['numero_niveau'].max().reset_index()
    df_final.drop_duplicates(inplace=True)
    df_final['annee'] = df_final['date'].dt.year
    df_final['mois'] = df_final['date'].dt.month

    df_final = df_final.groupby([filter_type, 'annee', 'numero_niveau'])['date'].count().reset_index(name='nbre_jour')
    
    df_final['nom_niveau'] = df_final['numero_niveau'].map(num_niveau_to_nom_niveau).fillna(np.nan)

    return df_final, df_return

def calculate_non_restriction_days(df_final_dataframe):
    
    total_days = df_final_dataframe.groupby([filter_type, 'annee'])['nbre_jour'].sum().reset_index()
    # adapt number of days since it depends on the year!
    total_days['non_restriction'] = 366 - total_days['nbre_jour']
    total_days.drop(columns=['nbre_jour'], inplace=True)
    total_days.rename(columns={'non_restriction': 'nbre_jour'}, inplace=True)
    total_days['numero_niveau'] = 0
    total_days['nom_niveau'] = 'Pas de restriction'

    return pd.concat([df_final_dataframe, total_days]).reset_index(drop=True)

## **Load and Preprocess Data**

In [57]:
df_corres_commune_insee = pd.read_csv("./correspondance-code-insee-code-postal.csv", sep=';')
df_corres_commune_insee = clean_data(df_corres_commune_insee)
df_corres_commune_insee.head()

,code_insee,code_postal,commune,departement,region
0,63402,63550,SAINT-VICTOR-MONTVIANEIX,PUY-DE-DOME,AUVERGNE
1,64012,64130,AINHARP,PYRENEES-ATLANTIQUES,AQUITAINE
2,26027,26150,BARSAC,DROME,RHONE-ALPES
3,2A108,20126,EVISA,CORSE-DU-SUD,CORSE
4,42205,42940,SAINT-BONNET-LE-COURREAU,LOIRE,RHONE-ALPES


In [58]:
df_zone_alerte_commune = pd.read_csv(zone_alerte_commune_url)
id_zone_to_code_commune = df_zone_alerte_commune.set_index('id_zone')['code_commune'].to_dict()
df_zone_alerte_commune.head()

,id_zone,code_commune
0,4091,26002
1,4091,26009
2,4091,26010
3,4091,26041
4,4091,26118


In [103]:
dataframes = {}
years = range(2010, 2024)

for year in years:
    arrete_url = f"https://www.data.gouv.fr/fr/datasets/r/{dataset_id_for_year[year]}"
    df_name = f'df_arrete_{year}'
    df = pd.read_csv(arrete_url, engine='python')
    dataframes[year] = df
    
dataframes[2010]['nom_niveau'].replace({"Absence de restriction": "Pas de restriction", "Crise renforcée": "Crise", "Crise modérée": "Alerte renforcée", "Arrêt des prélèvements non prioritaires": "Crise"} , inplace=True)
dataframes[2011]['nom_niveau'].replace({"Absence de restriction": "Pas de restriction", "Crise renforcée": "Crise", "Crise modérée": "Alerte renforcée", "Arrêt des prélèvements non prioritaires": "Crise", "Modification du régime hydraulique" : "Alerte"} , inplace=True)
nom_niveau_to_num_niveau = {"Pas de restriction": 0, "Vigilence": 1, "Alerte": 2, "Alerte renforcée": 3, "Crise": 4}
for year in years:
    dataframes[year]['numero_niveau'] = dataframes[year]['nom_niveau'].map(nom_niveau_to_num_niveau).fillna(np.nan)
    
# Assuming all necessary variables and dataframes are defined earlier
filter_type = 'departement'
df_final_dataframe, df_return = prepare_final_dataframe(dataframes, df_corres_commune_insee, id_zone_to_code_commune, filter_type)
df_final_dataframe = calculate_non_restriction_days(df_final_dataframe)
df_final_dataframe.sort_values('nbre_jour')

[nan]


,departement,annee,numero_niveau,nbre_jour,nom_niveau
1956,AIN,2020,0,0,Pas de restriction
1146,LOT,2013,2,1,Alerte
1957,AIN,2021,0,1,Pas de restriction
1708,TARN,2021,3,1,Alerte renforcée
355,CHARENTE-MARITIME,2022,1,1,Vigilance
...,...,...,...,...,...
2334,HAUTE-SAONE,2017,0,357,Pas de restriction
1629,SEINE-ET-MARNE,2011,4,360,Crise
2058,BOUCHES-DU-RHONE,2010,0,361,Pas de restriction
2576,MAYENNE,2015,0,363,Pas de restriction


In [105]:
niveau = df_final_dataframe.set_index('numero_niveau')['nom_niveau'].to_dict()
niveau

{2: 'Alerte',
 1: 'Vigilance',
 4: 'Crise',
 3: 'Alerte renforcée',
 0: 'Pas de restriction'}

## **Visualize Data**

In [106]:
data = df_final_dataframe.copy()

In [108]:
# Create a Dash web application
app = dash.Dash(__name__)

# Create a dropdown menu for selecting the department
departements = data[filter_type].unique()
departement_options = [{'label': departement, 'value': departement} for departement in departements]

# Create checkboxes for selecting nom_niveau values
niveaux = data['nom_niveau'].unique()
niveau_options = [{'label': niveau, 'value': niveau} for niveau in niveaux]

# Palette de 8 couleurs du clair au foncé
couleurs = ['#dddddd', '#bca12b', '#feb24c', '#fc4e2a', '#b10026']
couleur_map = {niveau[idx]: couleurs[idx] for idx in [0,1,2,3,4]}

# Layout of the dashboard
app.layout = html.Div([
    html.H1("Restriction de l'accès à l'eau"),
    
    # Dropdown menu to select the department
    dcc.Dropdown(
        id='departement-dropdown',
        options=departement_options,
        value=departements[0]  # Default value
    ),
    
    # Checkboxes to select nom_niveau values
    dcc.Checklist(
        id='niveau-checkboxes',
        options=niveau_options,
        value=[niveaux[0]]  # Default value
    ),
    
    # Stacked bar chart by year
    dcc.Graph(id='stacked-bar-chart')
    # Créer un graphique de ligne pour afficher l'évolution des restrictions par nom_niveau
])

# Callback to update the stacked bar chart based on the selected department and nom_niveau values
@app.callback(
    Output('stacked-bar-chart', 'figure'),
    [Input('departement-dropdown', 'value'),
     Input('niveau-checkboxes', 'value')]
)
def update_stacked_bar_chart(selected_departement, selected_niveaux):
    filtered_data = data[(data[filter_type] == selected_departement) & (data['nom_niveau'].isin(selected_niveaux))]
    
    # Create the stacked bar chart
    fig_stacked = px.bar(
        filtered_data,
        x='annee',
        y='nbre_jour',
        color='nom_niveau',
        color_discrete_map=couleur_map,
        title=f"Evolution du nombre de jours de restriction d'eau par niveau de crise pour le {filter_type} {selected_departement}"
    )
    
    return fig_stacked

# Run the application
if __name__ == '__main__':
    app.run_server(debug=True)